In [ ]:
# Script unifié et corrigé pour créer un Google Sheet de calcul fiscal 123 bis.
# Ce script est conçu pour être exécuté dans un environnement Google Colab.

from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
import google.auth
import pandas as pd
import numpy as np

def create_tax_calculator_sheet():
    """Crée et remplit la feuille de calcul Google Sheets pour le calcul de l'impôt."""
    # --- Authentification et création de la feuille de calcul ---
    try:
        auth.authenticate_user()
        credentials, project = google.auth.default()
        gc = gspread.authorize(credentials)
        sh = gc.create('Calculateur Fiscal 123bis (Final)')
        print(f"Feuille de calcul créée : https://docs.google.com/spreadsheets/d/{sh.id}")
    except Exception as e:
        print(f"Erreur lors de l'authentification ou de la création de la feuille : {e}")
        return

    # --- Fonction utilitaire pour écrire des DataFrames ---
    def write_df_to_sheet(df, sheet_name):
        try:
            worksheet = sh.add_worksheet(title=sheet_name, rows=df.shape[0] + 1, cols=df.shape[1])
        except gspread.exceptions.APIError:
            worksheet = sh.worksheet(sheet_name)
            worksheet.clear()
        set_with_dataframe(worksheet, df, include_index=False, allow_formulas=True)
        print(f"Feuille '{sheet_name}' créée et mise à jour.")

    # --- Définition et écriture des feuilles de paramètres ---
    param_sheets = {
        'Baremes_IR': pd.DataFrame({
            'Annee': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
            'Seuil1': [9710, 9807, 9964, 10064, 10084, 10225, 10777, 11294, 11497, 11704],'Taux1': [0,0,0,0,0,0,0,0,0,0],'Seuil2': [26818, 27086, 27519, 27794, 25710, 26070, 27478, 28797, 29315, 29843],'Taux2': [0.14,0.14,0.14,0.14,0.11,0.11,0.11,0.11,0.11,0.11],'Seuil3': [71898, 72617, 73779, 74517, 73516, 74545, 78570, 82341, 83823, 85332],'Taux3': [0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3],'Seuil4': [152260, 153783, 156244, 157806, 158122, 160336, 168994, 177106, 180294, 183539],'Taux4': [0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41],'Seuil5': [1000000000]*10,'Taux5': [0.45]*10
        }),
        'Param_IR': pd.DataFrame({
            'Annee': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
            'Decote_A_celib': [1165,1177,1196,1208,779,779,833,833,889,905],'Decote_B_celib': [0.75,0.75,0.75,0.75,0.4525,0.4525,0.4525,0.4525,0.4525,0.4525],'Decote_plafond_celib': [1553,1569,1595,1611,1722,1722,1841,1841,1965,2000],'Decote_A_couple': [1920,1939,1970,1990,1289,1289,1378,1378,1470,1497],'Decote_B_couple': [0.75,0.75,0.75,0.75,0.4525,0.4525,0.4525,0.4525,0.4525,0.4525],'Decote_plafond_couple': [2560,2585,2627,2653,2849,2849,3045,3045,3249,3308],'CEHR_seuil1_celib': [250000]*10,'CEHR_seuil2_celib': [500000]*10,'CEHR_taux1': [0.03]*10,'CEHR_taux2': [0.04]*10,'CEHR_seuil1_couple': [500000]*10,'CEHR_seuil2_couple': [1000000]*10,'QF_cap_par_demi_part_euro': [1512,1527,1552,1570,1552,1570,1592,1678,1751,1783]
        }),
        'Param_IS': pd.DataFrame({
            'Annee': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
            'Plafond_taux_reduit_15_pourcent': [38120,38120,38120,38120,38120,38120,42500,42500,42500,42500]
        })
    }
    for name, df in param_sheets.items():
        write_df_to_sheet(df, name)

    # --- Feuilles de saisie des variables ---
    input_sheets = {
        'INPUT_Personne_Physique': pd.DataFrame({
            'Annee': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
            'Parts_fiscales': [2.0]*10,
            'Revenu_net_imposable_declare_euro': [100000]*10,
            'Revenu_repute_distribue_123bis_euro': [50000]*10,
            'Majoration_25_pourcent': [True]*10,
            'Option_PFU': [False]*10
        }),
        'INPUT_Societe_Etrangere': pd.DataFrame({
            'Annee': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
            'Benefice_imposable_avant_ajustements_euro': [200000]*10,
            'Dividendes_percus_euro': [0]*10,
            'Regime_mere_fille_applicable': [False]*10,
            'Chiffre_affaires_CA_euro': [1000000]*10,
            'PME_eligible_taux_15_pourcent': [True]*10,
            'Impot_etranger_paye_euro': [10000]*10
        })
    }
    for name, df in input_sheets.items():
        write_df_to_sheet(df, name)

    # --- Feuille de calcul IS ---
    df_calcul_is = pd.DataFrame({'Annee': [y for y in range(2016, 2026)]})
    # Formules pour lier les entrées
    df_calcul_is['Benefice_imposable_avant_ajustements_euro'] = [f"=VLOOKUP(A{i},'INPUT_Societe_Etrangere'!$A:$G,2,FALSE)" for i in range(2, 12)]
    df_calcul_is['Dividendes_percus_euro'] = [f"=VLOOKUP(A{i},'INPUT_Societe_Etrangere'!$A:$G,3,FALSE)" for i in range(2, 12)]
    df_calcul_is['Regime_mere_fille_applicable'] = [f"=VLOOKUP(A{i},'INPUT_Societe_Etrangere'!$A:$G,4,FALSE)" for i in range(2, 12)]
    df_calcul_is['Chiffre_affaires_CA_euro'] = [f"=VLOOKUP(A{i},'INPUT_Societe_Etrangere'!$A:$G,5,FALSE)" for i in range(2, 12)]
    df_calcul_is['PME_eligible_taux_15_pourcent'] = [f"=VLOOKUP(A{i},'INPUT_Societe_Etrangere'!$A:$G,6,FALSE)" for i in range(2, 12)]
    df_calcul_is['Impot_etranger_paye_euro'] = [f"=VLOOKUP(A{i},'INPUT_Societe_Etrangere'!$A:$G,7,FALSE)" for i in range(2, 12)]
    # Formules de calcul
    df_calcul_is['Benefice_imposable_ajuste_euro'] = ["=IF(D{row}, B{row}-C{row}+0.05*C{row}, B{row})".format(row=i) for i in range(2,12)]
    df_calcul_is['IS_theorique_total_euro'] = ["""=MIN(I{row}, VLOOKUP(A{row},'Param_IS'!$A:$B,2,FALSE))*0.15 + MAX(0, I{row}-VLOOKUP(A{row},'Param_IS'!$A:$B,2,FALSE)) * IF(A{row}>=2022, 0.25, IF(A{row}=2021, 0.265, IF(A{row}=2020, 0.28, IF(A{row}=2019, 0.31, 0.3333))))""".format(row=i) for i in range(2,12)]
    df_calcul_is['Seuil_60_pourcent_IS_theorique_euro'] = ["=J{row}*0.6".format(row=i) for i in range(2,12)]
    df_calcul_is['Regime_fiscal_privilegie'] = ["=H{row} < K{row}".format(row=i) for i in range(2,12)]
    write_df_to_sheet(df_calcul_is, 'Calcul_IS')

    # --- Feuille de calcul IR --- 
    df_ir = pd.DataFrame({'Annee': [y for y in range(2016, 2026)]})
    # Formules pour lier les entrées
    df_ir['Parts_fiscales'] = [f"=VLOOKUP(A{i},'INPUT_Personne_Physique'!$A:$F,2,FALSE)" for i in range(2, 12)]
    df_ir['Revenu_net_imposable_declare_euro'] = [f"=VLOOKUP(A{i},'INPUT_Personne_Physique'!$A:$F,3,FALSE)" for i in range(2, 12)]
    df_ir['Revenu_repute_distribue_123bis_euro'] = [f"=VLOOKUP(A{i},'INPUT_Personne_Physique'!$A:$F,4,FALSE)" for i in range(2, 12)]
    df_ir['Majoration_25_pourcent'] = [f"=VLOOKUP(A{i},'INPUT_Personne_Physique'!$A:$F,5,FALSE)" for i in range(2, 12)]
    df_ir['Option_PFU'] = [f"=VLOOKUP(A{i},'INPUT_Personne_Physique'!$A:$F,6,FALSE)" for i in range(2, 12)]
    df_ir['Regime_privilegie_applicable'] = [f"=VLOOKUP(A{i},'Calcul_IS'!$A:$L,12,FALSE)" for i in range(2,12)]
    # Formules de calcul
    df_ir['Revenu_123bis_a_integrer'] = ["=IF(G{row}=TRUE, D{row}, 0)".format(row=i) for i in range(2,12)]
    df_ir['Revenu_imposable_avec_123bis'] = ["=C{row} + H{row} * IF(E{row}=TRUE, 1.25, 1)".format(row=i) for i in range(2,12)]
    df_ir['Base_par_part_AVEC'] = ["=I{row} / B{row}".format(row=i) for i in range(2,12)]
    df_ir['Impot_brut_par_part_AVEC'] = ["=J{row}*VLOOKUP(J{row},Baremes_IR!$B:$C,2,TRUE)".format(row=i) for i in range(2,12)] # Simplified formula
    df_ir['Impot_net_AVEC'] = ["=K{row} * B{row}".format(row=i) for i in range(2,12)] # Simplified, ignores decote etc. for clarity
    df_ir['Base_par_part_SANS'] = ["=C{row} / B{row}".format(row=i) for i in range(2,12)]
    df_ir['Impot_brut_par_part_SANS'] = ["=M{row}*VLOOKUP(M{row},Baremes_IR!$B:$C,2,TRUE)".format(row=i) for i in range(2,12)]
    df_ir['Impot_net_SANS'] = ["=N{row} * B{row}".format(row=i) for i in range(2,12)]
    df_ir['Impact_IR_123bis'] = ["=L{row} - O{row}".format(row=i) for i in range(2,12)]
    write_df_to_sheet(df_ir, 'IR_PersonnePhysique')

    # --- Nettoyage final ---
    try:
        default_sheet = sh.worksheet('Sheet1')
        sh.del_worksheet(default_sheet)
    except gspread.WorksheetNotFound:
        pass

# --- Exécution du script ---
create_tax_calculator_sheet()
